Analysing the deform convolution on COCO dataset

In [ ]:
!pip install fiftyone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 986.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
import torch
import torchvision
from torchvision.models.segmentation import deeplabv3_resnet50
from torchvision import transforms
from pycocotools.coco import COCO
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import fiftyone.zoo as foz
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights, lraspp_mobilenet_v3_large, LRASPP_MobileNet_V3_Large_Weights
from torchvision.transforms.functional import to_pil_image
from torchvision.models import resnet50, ResNet50_Weights, alexnet, AlexNet_Weights
from torchvision.ops import deform_conv2d
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torch.utils.data import Dataset
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
import PIL.Image
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image, to_grayscale, to_tensor


Migrating database to v0.23.1


INFO:fiftyone.migrations.runner:Migrating database to v0.23.1


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classes = ['bicycle', 'car', 'cat', 'chair', 'cow', 'dog', 'horse', 'person', 'sheep']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

# Define the CNN model
class SimpleCNNN(nn.Module):
    def __init__(self, classes):
        super(SimpleCNNN, self).__init__()
        self.conv1=nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1).to(device)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=16, stride=16)
        self.fc1 = nn.Linear(2 * 14 * 14, len(classes))

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = x.view(-1, 2 * 14 * 14)
        x = self.fc1(x)
        return x



class COCO(Dataset):
    def __init__(self, datasets, classes, transforms=None):
        self.dataset = datasets
        self.classes = classes
        self.transform = transforms



    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = Image.open(sample.filepath).convert('RGB')


        label = np.zeros(len(self.classes), dtype=np.float32)

        for detection in sample.ground_truth.detections:
            if detection.label in classes:
                label[classes.index(detection.label)] = 1.0
        image = self.transform(image)

        label = torch.tensor(label, dtype=torch.float64)

        return image, label


In [ ]:
dataset_train = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["segmentations"],
    classes=classes,
    max_samples=20000,
)
dataset_train = list(dataset_train)


dataset_test = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=classes,
    max_samples=2000,
)
dataset_test = list(dataset_test)



INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [2.8s elapsed, 0s remaining, 732.5Mb/s]      


INFO:eta.core.utils: 100% |██████|    1.9Gb/1.9Gb [2.8s elapsed, 0s remaining, 732.5Mb/s]      


Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_train2017.json'


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'validation'


 100% |███████████████| 2000/2000 [36.0s elapsed, 0s remaining, 45.1 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2000/2000 [36.0s elapsed, 0s remaining, 45.1 samples/s]      


Dataset 'coco-2017-validation-2000' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-validation-2000' created


In [ ]:
# Define the transformation
transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
trainset = COCO(datasets=dataset_train, classes=classes, transforms=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = COCO(datasets=dataset_test, classes=classes, transforms=transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


In [ ]:

model = SimpleCNNN(classes).to(device)



criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

epochs = 10

for epoch in range(epochs):

    running_loss = 0.0
    overall_accuracy = 0
    accuracy_per_label = torch.zeros(len(classes), device=device)
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        preds = torch.sigmoid(outputs) > 0.5
        correct_predictions = (preds == labels).float()

        accuracy_per_label += correct_predictions.sum(0)/(len(labels))

        overall_accuracy += correct_predictions.sum()/(len(labels)*(len(classes)))

    accuracy_per_label /= len(trainloader)
    running_loss /= len(trainloader)
    overall_accuracy /= len(trainloader)

    print(f'Train Loss: {running_loss:.4f} Total Acc: {overall_accuracy:.4f}')
    print('Per Class Acc:', accuracy_per_label.tolist())


print("Finished Training")



Train Loss: 0.2792 Total Acc: 0.9109
Per Class Acc: [0.9449641704559326, 0.8444964289665222, 0.9489451050758362, 0.8284733295440674, 0.9745720624923706, 0.9429737329483032, 0.9618332386016846, 0.7875199317932129, 0.9645203351974487]
Train Loss: 0.2552 Total Acc: 0.9174
Per Class Acc: [0.9604896903038025, 0.8443471193313599, 0.9486464858055115, 0.841610312461853, 0.9748706221580505, 0.9462579488754272, 0.9632762670516968, 0.7948347926139832, 0.982235312461853]
Train Loss: 0.2511 Total Acc: 0.9174
Per Class Acc: [0.9607882499694824, 0.8435012102127075, 0.9487957954406738, 0.841908872127533, 0.9753184914588928, 0.9459593892097473, 0.9626791477203369, 0.7951831221580505, 0.982533872127533]
Train Loss: 0.2491 Total Acc: 0.9175
Per Class Acc: [0.9607882499694824, 0.844894528388977, 0.9486464858055115, 0.8411624431610107, 0.9754677414894104, 0.9459593892097473, 0.9631270170211792, 0.7948845624923706, 0.9823845624923706]
Train Loss: 0.2480 Total Acc: 0.9175
Per Class Acc: [0.9604896903038025, 

In [ ]:


with torch.no_grad():

    overall_accuracy = 0
    accuracy_per_label = torch.zeros(len(classes), device=device)

    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        correct_predictions = (preds == labels).float()

        accuracy_per_label += correct_predictions.sum(0)/(len(labels))

        overall_accuracy += correct_predictions.sum()/(len(labels)*(len(classes)))

    accuracy_per_label /= len(testloader)

    overall_accuracy /= len(testloader)

    print(f'Total Acc: {overall_accuracy:.4f}')
    print('Per Class Acc:', accuracy_per_label.tolist())


Total Acc: 0.9131
Per Class Acc: [0.9523810148239136, 0.843254029750824, 0.9439484477043152, 0.8293651342391968, 0.9737103581428528, 0.9474207162857056, 0.9608135223388672, 0.785714328289032, 0.9816468954086304]


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False):
        super(DeformableConv2d, self).__init__()

        self.padding = padding

        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, stride=stride, padding=self.padding, bias=True).to(device)


        nn.init.kaiming_normal_(self.offset_conv.weight, nonlinearity='relu')
        nn.init.constant_(self.offset_conv.bias, 0.)


        self.modulator_conv = nn.Conv2d(in_channels, 1 * kernel_size * kernel_size,
                                        kernel_size=kernel_size, stride=stride, padding=self.padding, bias=True).to(device)

        nn.init.kaiming_normal_(self.modulator_conv.weight, nonlinearity='relu')
        nn.init.constant_(self.modulator_conv.bias, 0.)


        self.regular_conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                      kernel_size=kernel_size, stride=stride, padding=self.padding, bias=bias).to(device)

    def forward(self, x):
        h, w = x.shape[2:]
        max_offset = max(h, w)

        offset = self.offset_conv(x).clamp(-max_offset, max_offset).to(device)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x)).to(device)

        grid = self.generate_grid(offset, x.shape).to(device)

        x = F.grid_sample(x, grid, mode='bilinear', padding_mode='border')

        x = self.regular_conv(x).to(device)

        return x

    def generate_grid(self, offset, shape):
        """
        Generate a grid for deformable convolution
        """
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        batch_size, channel, height, width = shape

        h_range = torch.arange(0, height).view(1, height, 1).expand(batch_size, -1, width).to(device)
        w_range = torch.arange(0, width).view(1, 1, width).expand(batch_size, height, -1).to(device)

        offset = offset / max(height, width)
        offset = offset.permute(0, 2, 3, 1).contiguous().to(device)

        grid = torch.stack([w_range + offset[..., 0], h_range + offset[..., 1]], dim=-1).to(device)

        grid = 2.0 * grid / max(height, width) - 1.0

        return grid

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

class SimpleCNN(nn.Module):
    def __init__(self, classes):
        super(SimpleCNN, self).__init__()
        self.conv1=DeformableConv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1).to(device)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=16, stride=16)


        self.fc1 = nn.Linear(2 * 14 * 14, len(classes))

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = x.view(-1, 2 * 14 * 14)
        x = self.fc1(x)
        return x








In [ ]:

model = SimpleCNN(classes).to(device)



criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

epochs = 10

for epoch in range(epochs):

    running_loss = 0.0
    overall_accuracy = 0
    accuracy_per_label = torch.zeros(len(classes), device=device)
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        preds = torch.sigmoid(outputs) > 0.5
        correct_predictions = (preds == labels).float()

        accuracy_per_label += correct_predictions.sum(0)/(len(labels))

        overall_accuracy += correct_predictions.sum()/(len(labels)*(len(classes)))

    accuracy_per_label /= len(trainloader)
    running_loss /= len(trainloader)
    overall_accuracy /= len(trainloader)

    print(f'Train Loss: {running_loss:.4f} Total Acc: {overall_accuracy:.4f}')
    print('Per Class Acc:', accuracy_per_label.tolist())


print("Finished Training")



/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Train Loss: 0.2927 Total Acc: 0.9124
Per Class Acc: [0.9543192982673645, 0.8390724658966064, 0.9470541477203369, 0.8367834687232971, 0.970193088054657, 0.9382961988449097, 0.9622810482978821, 0.7865247130393982, 0.9768610596656799]
Train Loss: 0.2584 Total Acc: 0.9176
Per Class Acc: [0.9601910710334778, 0.8461385369300842, 0.9486464858055115, 0.8418591022491455, 0.9753184914588928, 0.9458101391792297, 0.9629777073860168, 0.7949841022491455, 0.982533872127533]
Train Loss: 0.2533 Total Acc: 0.9176
Per Class Acc: [0.9606389403343201, 0.8446457386016846, 0.9489451050758362, 0.8415107727050781, 0.9753184914588928, 0.9461086988449097, 0.9631270170211792, 0.7957802414894104, 0.9823845624923706]
Train Loss: 0.2500 Total Acc: 0.9173
Per Class Acc: [0.9606389403343201, 0.8438495397567749, 0.9484972357749939, 0.8406150341033936, 0.9753184914588928, 0.9458101391792297, 0.9632762670516968, 0.795033872127533, 0.9823845624923706]
Train Loss: 0.2485 Total Acc: 0.9175
Per Class Acc: [0.9606389403343201

In [ ]:


with torch.no_grad():

    overall_accuracy = 0
    accuracy_per_label = torch.zeros(len(classes), device=device)

    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        correct_predictions = (preds == labels).float()

        accuracy_per_label += correct_predictions.sum(0)/(len(labels))

        overall_accuracy += correct_predictions.sum()/(len(labels)*(len(classes)))

    accuracy_per_label /= len(testloader)

    overall_accuracy /= len(testloader)

    print(f'Total Acc: {overall_accuracy:.4f}')
    print('Per Class Acc:', accuracy_per_label.tolist())


Total Acc: 0.9131
Per Class Acc: [0.9523810148239136, 0.8437500596046448, 0.9434524178504944, 0.828869104385376, 0.9737103581428528, 0.9474207162857056, 0.9608135223388672, 0.7862103581428528, 0.9816468954086304]


In [ ]:
print(model)

SimpleCNN(
  (conv1): DeformableConv2d(
    (offset_conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (modulator_conv): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (regular_conv): Conv2d(3, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=392, out_features=9, bias=True)
)


Here we can see that in some classes the deform convolution has performed better than normal convolution

Analysing on Mnist dataset

In [4]:
import torch
import numpy as np
import random
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
    np.random.seed(seed)
    random.seed(seed)

In [5]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


import torch
from torch import nn


class MNISTClassifier(nn.Module):
    def __init__(self,
                 deformable=False):

        super(MNISTClassifier, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        conv = nn.Conv2d if deformable==False else DeformableConv2d
        self.conv4 = conv(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = conv(32, 32, kernel_size=3, stride=1, padding=1, bias=True)

        self.pool = nn.MaxPool2d(2)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x) # [14, 14]
        x = torch.relu(self.conv2(x))
        x = self.pool(x) # [7, 7]
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = self.gap(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x


def train(model, loss_function, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()


def test(model, loss_function, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    num_data = 0
    with torch.no_grad():
        for data, target in test_loader:
            org_data, target = data.to(device), target.to(device)

            for scale in np.arange(0.5, 1.6, 0.1): # [0.5, 0.6, ... ,1.2, 1.3, 1.4, 1.5]
                data = transforms.functional.affine(org_data, scale=scale, angle=0, translate=[0,0],shear=0)
                output = model(data)
                test_loss += loss_function(output, target).item()  # sum up batch mean loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
                num_data += len(data)

    test_loss /= num_data

    test_acc = 100. * correct / num_data
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, num_data,
        test_acc))
    return test_acc


def main(use_deformable_conv=False):
    # Training settings
    seed=1
    setup_seed(seed)

    use_cuda = torch.cuda.is_available()
    batch_size = 64
    lr=1e-3
    gamma=0.7
    epochs=14

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 2,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    train_transform = transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

    dataset1 = datasets.MNIST('./data', train=True, download=True,
                       transform=train_transform)
    dataset2 = datasets.MNIST('./data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = MNISTClassifier(use_deformable_conv).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    loss_function = nn.CrossEntropyLoss()
    best_test_acc = 0.
    for epoch in range(1, epochs + 1):
        train(model, loss_function, device, train_loader, optimizer, epoch)
        best_test_acc = max(best_test_acc, test(model, loss_function, device, test_loader))
        scheduler.step()
    print("best top1 acc(%): ", f"{best_test_acc:.2f}")

In [6]:
main(use_deformable_conv=False)


Test set: Average loss: 0.0107, Accuracy: 88585/110000 (80.53%)


Test set: Average loss: 0.0080, Accuracy: 92462/110000 (84.06%)


Test set: Average loss: 0.0075, Accuracy: 93799/110000 (85.27%)


Test set: Average loss: 0.0065, Accuracy: 95916/110000 (87.20%)


Test set: Average loss: 0.0071, Accuracy: 95237/110000 (86.58%)


Test set: Average loss: 0.0062, Accuracy: 96837/110000 (88.03%)


Test set: Average loss: 0.0059, Accuracy: 97318/110000 (88.47%)


Test set: Average loss: 0.0056, Accuracy: 97927/110000 (89.02%)


Test set: Average loss: 0.0059, Accuracy: 97830/110000 (88.94%)


Test set: Average loss: 0.0060, Accuracy: 97353/110000 (88.50%)


Test set: Average loss: 0.0059, Accuracy: 97886/110000 (88.99%)


Test set: Average loss: 0.0059, Accuracy: 97700/110000 (88.82%)


Test set: Average loss: 0.0060, Accuracy: 97577/110000 (88.71%)


Test set: Average loss: 0.0060, Accuracy: 97624/110000 (88.75%)

best top1 acc(%):  89.02


In [6]:
main(use_deformable_conv=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 428624417.45it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 28435608.88it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 251285931.13it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21550371.91it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(



Test set: Average loss: 0.0091, Accuracy: 90412/110000 (82.19%)


Test set: Average loss: 0.0072, Accuracy: 93704/110000 (85.19%)


Test set: Average loss: 0.0058, Accuracy: 97373/110000 (88.52%)


Test set: Average loss: 0.0059, Accuracy: 96785/110000 (87.99%)


Test set: Average loss: 0.0052, Accuracy: 98408/110000 (89.46%)


Test set: Average loss: 0.0048, Accuracy: 99407/110000 (90.37%)


Test set: Average loss: 0.0050, Accuracy: 98791/110000 (89.81%)


Test set: Average loss: 0.0048, Accuracy: 99381/110000 (90.35%)


Test set: Average loss: 0.0046, Accuracy: 99730/110000 (90.66%)


Test set: Average loss: 0.0047, Accuracy: 99427/110000 (90.39%)


Test set: Average loss: 0.0047, Accuracy: 99474/110000 (90.43%)


Test set: Average loss: 0.0048, Accuracy: 99273/110000 (90.25%)


Test set: Average loss: 0.0046, Accuracy: 99608/110000 (90.55%)


Test set: Average loss: 0.0046, Accuracy: 99707/110000 (90.64%)

best top1 acc(%):  90.66


Here we can also see that deform convolution has slightly performed better

Analysing on CIFAR10 dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if GPU (CUDA) is available, otherwise, use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Load CIFAR-10 dataset



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Instantiate the model
model = SimpleCNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
epochs = 40

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
            running_loss = 0.0

print("Finished Training")

# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Using downloaded and verified file: ./data/cifar-10-python.tar.gz
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Finished Training
Accuracy on the test set: 74.32%


In [ ]:
# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Accuracy on the test set: 100.00%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if GPU (CUDA) is available, otherwise, use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class SimpleCNNN(nn.Module):
    def __init__(self):
        super(SimpleCNNN, self).__init__()
        #self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv1=DeformableConv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1).to(device)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        #self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv2=DeformableConv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1).to(device)

        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Load CIFAR-10 dataset



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Instantiate the model
model = SimpleCNNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
epochs = 40

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
            running_loss = 0.0

print("Finished Training")

# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")
# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

100%|██████████| 170498071/170498071 [00:03<00:00, 47665745.39it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Finished Training
Accuracy on the test set: 71.48%
Accuracy on the test set: 100.00%


در اینجا البته دفورم کانولوشن بهتر عمل نمی کند و یک اختلاف جزیی به دلیل نویز دارند



و در کل در هر کدام از دیتاست ها یا دفورم کانولوشن کمی بهتر عمل کرده یا اختلاف جزیی با کانولوشن معمولی دارد


و زمان طول کشیدن یادگیری شبکه با دفورم کانولوشن تا حدی بیشتر از کانولوشن معمولی بود زیرا خود دفورم کانولوشن شامل چندین محاسبه و کانولوشن معمولی می باشد

#Q1


Let's explore key distinctions in grid sampling:

Standard Convolution:
Fixed Sampling Grid: During the sampling process, convolutional filters adhere to a fixed grid pattern.
Limited Flexibility: Standard convolutions demonstrate lower adaptability to changes and deformations in the input data.
Deformable Convolution:
Flexible Sampling Grid: Convolutional filters dynamically alter their sampling grid based on the input content.
Enhanced Adaptability: Deformable convolutions excel in capturing deformable structures and intricate patterns in the data.
A Comparative Analysis:
Handling Complex Patterns: Deformable convolutions stand out in capturing intricate patterns and structures that may not conform to a regular grid.
Adaptability Boost: The adaptive grid in deformable convolutions enables superior handling of variations in object shapes and positions.
Performance Enhancement: In tasks involving object deformations, deformable convolutions often result in superior performance compared to standard convolutions.

#Q2



Let's delve into how Deformable Convolutional Networks foster flexibility amid geometric transformations:

Dynamic Offsets Learning:

Within deformable convolutional layers, the network introduces learnable offsets rather than relying on a fixed sampling grid.
These offsets, additional parameters learned during training, dictate how convolutional filters sample input values, facilitating adaptation to geometric transformations.
Adaptation of Spatial Sampling Grid:

Deformable convolutions empower each location in the feature map to possess its own sampling grid.
The sampling grid adjusts based on learned offsets, enabling the network to prioritize relevant regions and adapt to spatial transformations.
Precision in Feature Localization:

Deformable convolutions empower the network to pinpoint features more accurately, especially in the presence of deformations and variations in object shapes.
Learnable offsets aid the network in focusing on informative regions, enhancing the capture of geometrically transformed patterns.
Refined Object Localization:

Traditional convolutional layers might face challenges in accurately localizing objects amid geometric transformations.
Deformable convolutions enhance object localization by enabling the network to dynamically adjust its receptive field.
Augmented Robustness:

By permitting convolutional filters to adapt to the specific content of input data, deformable convolutions bolster the network's robustness to geometric transformations.
This adaptability proves particularly advantageous in tasks where objects may undergo diverse deformations or changes in appearance.

#Q3




Let's examine the hurdles faced by standard convolutional layers when confronted with images featuring objects undergoing substantial spatial changes, such as rotations or deformations. Here are some explanations for why conventional convolutional layers might struggle in handling such scenarios:

Rigidity in Sampling:

Standard convolutions employ a fixed grid for sampling input values.
This fixed grid may prove inadequate for capturing spatial variations or deformations in the input, particularly when objects undergo significant transformations.
Constricted Receptive Field:

Convolutional layers possess a limited receptive field, focusing on a local region of the input at a time.
In instances of spatial transformations, standard convolutional layers may lack a sufficiently expansive receptive field to encompass the entire transformed object.
Absence of Adaptability:

Simple convolutional layers lack the capacity to adapt to the specific content of input data.
When confronted with geometric transformations, a fixed convolutional grid may struggle to align with deformed structures, resulting in suboptimal feature extraction.
Spatial Information Erosion:

Rotation or deformation can lead to a loss of spatial information if the convolutional layer's receptive field is insufficiently large.
Standard convolutions may face challenges in preserving spatial relationships between pixels, compromising the network's ability to comprehend transformed objects.
Limited Transformation Handling:

Simple convolutional layers are tailored for translation invariance but may falter in managing more intricate spatial transformations.
While translation invariance is advantageous, it may prove insufficient when confronted with rotated or deformed objects.
Localization Challenges:

Standard convolutions may encounter difficulty in precisely localizing objects undergoing spatial changes.
The fixed nature of the convolutional grid may contribute to imprecise object localization amid substantial transformations.

#Q4



In Deformable Convolutional Networks (DCNs), the offsets represent learnable parameters computed during the training process via backpropagation.